# Feature Engineering

In this notebook, we will create new features from the existing data to enhance the predictive power of our models.


In [12]:
# Importing necessary libraries
import pandas as pd
import numpy as np


In [13]:
# Loading the dataset
data = pd.read_csv('./data/cleaned_weekly_stock_market.csv')
data.head()


,companyName,date,openingPrice,highestPrice,lowestPrice,closingPrice,volume
0,AMEN BANK,2014-06-16,23.63,23.63,22.75,23.14,1608.0
1,AMEN BANK,2014-06-23,23.14,23.14,22.37,22.75,16837.0
2,AMEN BANK,2014-06-30,22.75,22.97,22.07,22.66,33514.0
3,AMEN BANK,2014-07-07,22.75,23.17,21.88,22.74,3340.0
4,AMEN BANK,2014-07-14,22.84,23.16,22.58,22.75,5789.0


In [14]:
# Extracting date features
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day_of_month'] = data['date'].dt.day
data['week_of_year'] = data['date'].dt.isocalendar().week

data.head()

,companyName,date,openingPrice,highestPrice,lowestPrice,closingPrice,volume,year,month,day_of_month,week_of_year
0,AMEN BANK,2014-06-16,23.63,23.63,22.75,23.14,1608.0,2014,6,16,25
1,AMEN BANK,2014-06-23,23.14,23.14,22.37,22.75,16837.0,2014,6,23,26
2,AMEN BANK,2014-06-30,22.75,22.97,22.07,22.66,33514.0,2014,6,30,27
3,AMEN BANK,2014-07-07,22.75,23.17,21.88,22.74,3340.0,2014,7,7,28
4,AMEN BANK,2014-07-14,22.84,23.16,22.58,22.75,5789.0,2014,7,14,29


In [15]:
# Extracting Price features
data['price_range'] = data['highestPrice'] - data['lowestPrice']
data['price_change'] = data ['closingPrice'] - data['openingPrice']
data['weekly_return'] = (data['price_change'] / data['openingPrice']) * 100

# Log transformation of volume to handle skewness
data['log_volume'] = np.log1p(data['volume'])

data.head()

,companyName,date,openingPrice,highestPrice,lowestPrice,closingPrice,volume,year,month,day_of_month,week_of_year,price_range,price_change,weekly_return,log_volume
0,AMEN BANK,2014-06-16,23.63,23.63,22.75,23.14,1608.0,2014,6,16,25,0.88,-0.49,-2.073635,7.383368
1,AMEN BANK,2014-06-23,23.14,23.14,22.37,22.75,16837.0,2014,6,23,26,0.77,-0.39,-1.685393,9.731394
2,AMEN BANK,2014-06-30,22.75,22.97,22.07,22.66,33514.0,2014,6,30,27,0.90,-0.09,-0.395604,10.419748
3,AMEN BANK,2014-07-07,22.75,23.17,21.88,22.74,3340.0,2014,7,7,28,1.29,-0.01,-0.043956,8.114025
4,AMEN BANK,2014-07-14,22.84,23.16,22.58,22.75,5789.0,2014,7,14,29,0.58,-0.09,-0.394046,8.663888


In [17]:
# Group by companyName to calculate moving averages and volatility for each company separately
data['moving_avg_4'] = data.groupby('companyName')['closingPrice'].transform(lambda x: x.rolling(window=4).mean())
data['ema_4'] = data.groupby('companyName')['closingPrice'].transform(lambda x: x.ewm(span=4, adjust=False).mean())
data['volatility_4'] = data.groupby('companyName')['closingPrice'].transform(lambda x: x.rolling(window=4).std())

print(data.head())

  companyName       date  openingPrice  highestPrice  lowestPrice  \
0   AMEN BANK 2014-06-16         23.63         23.63        22.75   
1   AMEN BANK 2014-06-23         23.14         23.14        22.37   
2   AMEN BANK 2014-06-30         22.75         22.97        22.07   
3   AMEN BANK 2014-07-07         22.75         23.17        21.88   
4   AMEN BANK 2014-07-14         22.84         23.16        22.58   

   closingPrice   volume  year  month  day_of_month  week_of_year  \
0         23.14   1608.0  2014      6            16            25   
1         22.75  16837.0  2014      6            23            26   
2         22.66  33514.0  2014      6            30            27   
3         22.74   3340.0  2014      7             7            28   
4         22.75   5789.0  2014      7            14            29   

   price_range  price_change  weekly_return  log_volume  moving_avg_4  \
0         0.88         -0.49      -2.073635    7.383368           NaN   
1         0.77         -

In [18]:
# Handling NaN values
data['moving_avg_4'].fillna(0, inplace=True)
data['volatility_4'].fillna(0, inplace=True)

C:\Users\Kahla\AppData\Local\Temp\ipykernel_5184\4065620996.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['moving_avg_4'].fillna(0, inplace=True)
C:\Users\Kahla\AppData\Local\Temp\ipykernel_5184\4065620996.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 